In [1]:
from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
%matplotlib inline
# code to regulate the size of plots for the rest of notebook
plt.rc('figure', figsize=(14, 12))
plt.rc('font', size=13)

In [2]:
colnames=['timestamp', 'time', 'destination', 'user', 'cohort',
          'ip', 'huh']
df = pd.read_csv('curriculum-access.txt',          
                 engine='python',
                 header=None,
                 index_col=False,
                 names=colnames,
                 error_bad_lines=False,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"')

In [3]:
df

,timestamp,time,destination,user,cohort,ip,huh
0,2018-01-26,09:55:03,/,1,8.0,98.106.20.62,NaN
1,2018-01-26,09:56:02,java-ii,1,8.0,98.106.20.62,NaN
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,98.106.20.62,NaN
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,98.106.20.62,NaN
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,98.106.20.62,NaN
...,...,...,...,...,...,...,...
543591,2020-05-15,15:01:40,java-ii/file-io,602,56.0,48.188.241.68,NaN
543592,2020-05-15,15:01:54,9-timeseries/3-prep,582,55.0,48.135.163.38,NaN
543593,2020-05-15,15:03:37,jquery/ajax/requests-and-responses,637,57.0,71.121.18.17,NaN
543594,2020-05-15,15:05:14,10-anomaly-detection/3-discrete-probabilistic-...,11,28.0,77.186.132.227,NaN


In [4]:
# df['Timestamp'] = df['timestamp'] + " " + df['time']
# df = df.drop(columns = ['timestamp', 'time', 'huh'])
# df['Timestamp'] = pd.to_datetime(df['Timestamp'])
# df = df.sort_values("Timestamp")
# df.head()

## Let's do some anomaly detection column by column

### Destination Column:
- Get a look at the data
> - Are there any nulls?
> - Are there any unexpected values?
> - What are the most and least frequently visited pages?

In [5]:
# get the observations where the nulls are occured
df.loc[df.destination.isnull()]

,timestamp,time,destination,user,cohort,ip,huh
506305,2020-04-08,09:25:18,NaN,586,55.0,73.178.241.52,NaN


In [6]:
# drop this column
df.drop(df.index[[506305]], inplace=True)

In [7]:
df.destination.value_counts()

/                                              28920
javascript-i                                   10950
search/search_index.json                       10859
toc                                             9839
html-css                                        8202
                                               ...  
,%20https://github.com/RaulCPena                   1
4.2-compare-means                                  1
html-css/positioning                               1
content/php_iii/alternative-syntax.html            1
content/examples/php/internal_functions.php        1
Name: destination, Length: 1811, dtype: int64

In [8]:
df.groupby(['destination', 'cohort']).user.value_counts()

destination                       cohort  user
%20https://github.com/RaulCPena   55.0    580       1
,%20https://github.com/RaulCPena  55.0    580       1
.git                              24.0    203       1
.gitignore                        24.0    203       1
/                                 1.0     11      237
                                                 ... 
web-design/ux/purpose             57.0    643       2
                                          627       1
                                          640       1
wp-admin                          22.0    180       1
wp-login                          22.0    180       1
Name: user, Length: 72039, dtype: int64

- Can you label students who are viewing both the web dev and data science curriculum? 
- Can you label students by the program they are in? Can you label users by student vs. staff? 
- What are Zach, Maggie, David, and Ryan's ids?
- Can you label students who are viewing both the web dev and data science curriculum? 
- Can you label students by the program they are in?
- Can you label users by student vs. staff?

In [9]:
df.user.nunique()

650

In [10]:
web_dev = df[df['destination'].str.contains('web')]
web_dev

,timestamp,time,destination,user,cohort,ip,huh
4831,2018-02-03,10:52:29,appendix/postwork/website-status-page,19,22.0,97.9.151.24,NaN
5218,2018-02-05,07:49:33,appendix/postwork/website-status-page,9,22.0,98.106.20.62,NaN
6972,2018-02-07,19:12:33,appendix/postwork/website-status-page,36,21.0,174.175.160.46,NaN
9136,2018-02-13,00:08:19,appendix/postwork/website-status-page,4,22.0,71.124.232.212,NaN
11436,2018-02-19,13:09:13,appendix/postwork/website-status-page,78,NaN,98.106.20.62,NaN
...,...,...,...,...,...,...,...
537503,2020-05-08,12:36:45,web-design/ui/visuals,524,52.0,109.200.118.236,NaN
537504,2020-05-08,12:36:46,web-design/ux/purpose,524,52.0,109.200.118.236,NaN
537505,2020-05-08,12:36:48,web-design/ux/layout,524,52.0,109.200.118.236,NaN
538760,2020-05-11,08:59:01,web-design/intro,641,57.0,68.12.212.116,NaN


In [11]:
def get_topic(destination):
    ds_list = ["python", "time series", "nlp"]
    web_dev = ["javascript", "java", "css"]
    for ds in ds_list:
        if ds in destination:
            return "Data Science"
    for wd in web_dev:
        if wd in destination:
            return "Web Dev"
    return "Unknown"
df["course"] = df.destination.apply(get_topic)

In [12]:
df.course.value_counts()

Unknown         269982
Web Dev         266456
Data Science      7157
Name: course, dtype: int64

In [13]:
df.destination.to_list()

['/',
 'java-ii',
 'java-ii/object-oriented-programming',
 'slides/object_oriented_programming',
 'javascript-i/conditionals',
 'javascript-i/loops',
 'javascript-i/conditionals',
 'javascript-i/functions',
 'javascript-i/loops',
 'javascript-i/functions',
 'mkdocs/search_index.json',
 'javascript-i/introduction/working-with-data-types-operators-and-variables',
 '/',
 'javascript-i',
 'javascript-i/introduction/working-with-data-types-operators-and-variables',
 'javascript-i/introduction/variables',
 'javascript-i/introduction/operators',
 'javascript-i/introduction/working-with-data-types-operators-and-variables',
 'javascript-i/functions',
 'javascript-i/functions',
 'javascript-i/functions',
 'javascript-i/conditionals',
 'javascript-i/functions',
 'mkdocs/search_index.json',
 'git/merge-conflict-demo',
 'mkdocs/search_index.json',
 'git/working-in-teams',
 'javascript-i/introduction/primitive-types',
 'javascript-i/introduction/operators',
 'javascript-i/functions',
 '/',
 'javascr

In [14]:
df.loc[df.destination == 'content/javascript']

,timestamp,time,destination,user,cohort,ip,huh,course
684,2018-01-27,15:56:13,content/javascript,51,13.0,73.180.162.40,NaN,Web Dev
691,2018-01-27,15:57:30,content/javascript,51,13.0,73.180.162.40,NaN,Web Dev
702,2018-01-27,16:09:05,content/javascript,51,13.0,73.180.162.40,NaN,Web Dev
4719,2018-02-02,22:50:59,content/javascript,80,14.0,137.51.30.194,NaN,Web Dev
4750,2018-02-03,00:07:55,content/javascript,51,13.0,73.180.162.40,NaN,Web Dev
...,...,...,...,...,...,...,...,...
440815,2020-01-28,11:23:16,content/javascript,545,28.0,13.107.209.195,NaN,Web Dev
519268,2020-04-20,21:01:01,content/javascript,646,8.0,174.175.198.147,NaN,Web Dev
521909,2020-04-23,09:56:06,content/javascript,646,8.0,174.175.198.147,NaN,Web Dev
521947,2020-04-23,10:12:44,content/javascript,646,8.0,174.175.198.147,NaN,Web Dev


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543595 entries, 0 to 543595
Data columns (total 8 columns):
timestamp      543595 non-null object
time           543595 non-null object
destination    543595 non-null object
user           543595 non-null int64
cohort         515739 non-null float64
ip             543595 non-null object
huh            0 non-null float64
course         543595 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 37.3+ MB


In [16]:
df.cohort.nunique()

36

In [17]:
df.loc[df.course == 'Data Science']

,timestamp,time,destination,user,cohort,ip,huh,course
217602,2019-03-12,09:17:47,ada-python-assessment.html,366,NaN,98.106.20.59,NaN,Data Science
218891,2019-03-14,08:50:02,ada-python-assessment.html,349,NaN,98.106.20.59,NaN,Data Science
218895,2019-03-14,08:51:04,ada-python-assessment.html,349,NaN,98.106.20.59,NaN,Data Science
218897,2019-03-14,08:55:27,ada-python-assessment.html,349,NaN,98.106.20.59,NaN,Data Science
219534,2019-03-14,18:59:45,ada-python-assessment.html,351,NaN,174.174.115.112,NaN,Data Science
...,...,...,...,...,...,...,...,...
542660,2020-05-14,17:00:51,4-python/8.3-intro-to-numpy,617,55.0,65.251.10.182,NaN,Data Science
542662,2020-05-14,17:01:34,4-python/8.4.1-pandas-overview,617,55.0,65.251.10.182,NaN,Data Science
542663,2020-05-14,17:02:40,4-python/8.4.2-series,617,55.0,65.251.10.182,NaN,Data Science
542709,2020-05-14,18:30:46,4-python/8.4.3-dataframes,617,55.0,65.251.10.182,NaN,Data Science


In [18]:
ip_df = pd.DataFrame(df.ip.value_counts(dropna=False)).reset_index().\
                rename(index=str, columns={'index': 'ip', 'ip': 'ip_count'})
ip_df2 = pd.DataFrame(df.ip.value_counts(dropna=False)/df.ip.count()).reset_index().\
                rename(index=str, columns={'index': 'ip', 'ip': 'ip_proba'})
ip_df = ip_df.merge(ip_df2)


# see those where rate < 1% 
ip_df[ip_df.ip_proba < .01]

,ip,ip_count,ip_proba
3,13.107.209.195,4219,0.007761
4,173.125.68.94,2980,0.005482
5,77.186.132.227,1826,0.003359
6,137.51.33.152,1788,0.003289
7,68.12.137.147,1614,0.002969
...,...,...,...
3490,108.78.220.35,1,0.000002
3491,65.158.242.247,1,0.000002
3492,173.59.110.81,1,0.000002
3493,69.202.241.18,1,0.000002
